In [32]:
import numpy as np
import random

random.seed(10)

In [42]:
# Hidden state space
S = [0, 1]
# Emission space
V = [1, 2, 3, 4, 5]

# Transition matrix
A = np.matrix([[0.8, 0.2], 
               [0.1, 0.9]])
# Initial distribution 
mu_0 = [0.5, 0.5]
# Emission matrix
B = np.matrix([[0.2, 0.5, 0.2, 0.1, 0],
               [0, 0.1, 0.4, 0.4, 0.1]])


In [89]:
class HMM():
    """
    S: state space
    V: emission space
    A: transition matrix
    mu_0: initial distribution
    B: emission matrix
    N: markov chain length
    """
    
    def __init__(self, S, V, A, mu_0, B, N):
        self.S = S
        self.V = V
        self.A = A
        self.mu_0 = mu_0
        self.B = B
        self.N = N
        
        self.markov_chain = self.markovChain()
        self.emission = self.emitStates()
        
    def markovChain(self):
        # Create markov chain
        chain = np.zeros(self.N)
        # Set the first state by sampling from the initial distribution
        chain[0] = np.random.choice(self.S, p = self.mu_0)
        
        # Set the remaining states in the chain
        for i in range(1, self.N):
            # Get the probability row from the transition matrix
            state_prob = np.array(self.A[int(chain[i - 1])]).flatten()
            # Sample from the state space
            chain[i] = np.random.choice(self.S, p = state_prob)
        
        return(chain)
    
    def emitStates(self):
        emission = np.zeros(self.N)
        
        for i in range(self.N):
            # Get the probability row from the emission matrix
            emit_prob = np.array(self.B[int(self.markov_chain[i - 1])]).flatten()
            # Sample from the emission matrix
            emission[i] = np.random.choice(self.V, p = emit_prob)
            
        return(emission)
        
hmm = HMM(S, V, A, mu_0, B, 115)
hmm.emission

array([4., 1., 1., 2., 1., 3., 4., 3., 5., 5., 3., 3., 3., 4., 3., 2., 3.,
       3., 1., 3., 1., 2., 2., 4., 1., 2., 2., 3., 3., 4., 3., 2., 4., 5.,
       4., 2., 4., 4., 3., 4., 4., 3., 4., 4., 2., 1., 3., 3., 4., 3., 5.,
       3., 4., 3., 3., 4., 2., 2., 2., 2., 2., 3., 2., 2., 1., 2., 2., 4.,
       2., 2., 2., 2., 2., 3., 3., 2., 3., 2., 3., 3., 2., 2., 2., 2., 1.,
       2., 4., 2., 2., 2., 2., 2., 1., 3., 4., 3., 2., 3., 3., 4., 4., 4.,
       4., 2., 3., 2., 4., 2., 3., 4., 4., 4., 3., 4., 4.])